<img src="https://toppng.com/uploads/preview/linkedin-logo-png-photo-116602552293wtc4qogql.png" width="20" height="20" /> [Bharath Hemachandran](https://www.linkedin.com/in/bharath-hemachandran/)

# 🧩 Phase 3: Multi-Step Agent with MCP

Same MCP (Hugging Face), but a **multi-step** prompt: the model will make **several tool calls** in one run (e.g. search, then summarize). No approval—<code>require_approval: "never"</code>.

<div style="background: #e1f5fe; padding: 14px; border-radius: 8px; border-left: 4px solid #0288d1;">
<strong>🎯 What you'll do:</strong> Ask the model to search Hugging Face for trending models and then summarize the top 2. You'll see multiple <code>mcp_call</code> steps in the output.
</div>

### 📋 Notebook objective (table of contents)

This notebook covers:
- **Setup** — OpenAI client and API keys (GROQ + optional HF_TOKEN)
- **MCP tool** — Same Hugging Face MCP config as Phase 2
- **Multi-step prompt** — Request that triggers multiple tool calls in one run
- **API call** — Groq Responses API with tools; model searches then summarizes
- **mcp_call steps** — Inspecting each tool-call item in <code>response.output</code>
- **Additional reading** — MCP and agent workflows


## 🔧 Setup (run once)

Same as Phase 2: **openai** and API keys (GROQ + optional HF_TOKEN).

In [ ]:
!pip install -q openai

In [ ]:
import os
from getpass import getpass

if not os.environ.get("GROQ_API_KEY"):
    os.environ["GROQ_API_KEY"] = getpass("Paste your GROQ_API_KEY: ")
if not os.environ.get("HF_TOKEN"):
    tok = getpass("Paste your HF_TOKEN (or press Enter to skip): ")
    if tok:
        os.environ["HF_TOKEN"] = tok

from openai import OpenAI

def get_groq_client():
    return OpenAI(
        api_key=os.environ["GROQ_API_KEY"],
        base_url="https://api.groq.com/openai/v1",
    )

HF_TOKEN = os.environ.get("HF_TOKEN")
print("✅ Groq client ready.")

## 🔧 MCP tool (same as Phase 2)

In [ ]:
mcp_tool = {
    "type": "mcp",
    "server_label": "Huggingface",
    "server_url": "https://huggingface.co/mcp",
    "server_description": "Search and access AI models from Hugging Face",
    "require_approval": "never",
}
if HF_TOKEN:
    mcp_tool["headers"] = {"Authorization": f"Bearer {HF_TOKEN}"}

print("MCP tool: Hugging Face")

## 🚀 Multi-step prompt

The prompt asks for **two steps**: (1) search for trending text-generation models, (2) summarize the top 2. The model may issue **multiple** <code>mcp_call</code> items before returning the final text.

In [ ]:
client = get_groq_client()

response = client.responses.create(
    model="openai/gpt-oss-120b",
    input=(
        "Search Hugging Face for trending text-generation models. "
        "Then summarize the top 2 in one short paragraph."
    ),
    tools=[mcp_tool],
)

print("--- output_text ---")
print(response.output_text)

## 📋 mcp_call steps

Each tool call appears as an item with <code>type == "mcp_call"</code>. Here we print a short preview of each.

In [ ]:
print("--- mcp_call steps ---")
for item in response.output:
    if getattr(item, "type", None) == "mcp_call":
        name = getattr(item, "name", "?")
        out = str(getattr(item, "output", ""))[:200]
        print(f"  {name}: {out}...")

In [ ]:
print("✅ Phase 3 complete. Next: build your own MCP or use Cursor agents.")

## 📚 Additional reading

**YouTube (verified)**  
- [The Model Context Protocol (MCP)](https://www.youtube.com/watch?v=CQywdSdi5iA) — Anthropic: MCP and tool integration.  
- [Model Context Protocol, clearly explained](https://www.youtube.com/watch?v=7j_NE6Pjv-E) — Multi-step agents and external tools.

**Blogs (popular)**  
- [Model Context Protocol](https://modelcontextprotocol.io) — Official site: protocol, servers, clients.  
- [Groq Blog – MCP and connectors](https://groq.com/blog) — Groq: MCP support and agent workflows.